In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from TextHandler import TextHandler

/opt/anaconda3/envs/CustomEnvironments/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pré-processamento de dados e definição de modelos

Classe TextHandler: Responsável pelo pré-processamento de texto.

In [2]:
txt_handler = TextHandler() 

In [3]:
df = pd.read_csv('reviews-pt-br.csv')
df.head(10)

,codigo,texto,sentimento
0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg
1,2,O Bravo parece indicar que o personagem princi...,neg
2,3,"Durante a Guerra pela Independência do Sul, GE...",pos
3,4,É fora de questão que a verdadeira Anna Anders...,pos
4,5,Concordo totalmente com outro dos revisores aq...,neg
5,6,Obra-prima absoluta de um filme! Boa noite Mr....,pos
6,7,Embora a palavra megalmania seja muito usada p...,pos
7,8,Esta tem que ser a peça mais incrível de porca...,neg
8,9,Eu suponho que todas as piadas internas são o ...,neg
9,10,"Se há um tema deste filme, é que as pessoas po...",pos


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
(df.sentimento.value_counts() / df.shape[0]) * 100

neg    50.112324
pos    49.887676
Name: sentimento, dtype: float64

Transformação da classe de sentimento para digitos binários (0, 1)

In [6]:
df['sentimento_codificado'] = le.fit_transform(df.sentimento)

In [7]:
df.head()

,codigo,texto,sentimento,sentimento_codificado
0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg,0
1,2,O Bravo parece indicar que o personagem princi...,neg,0
2,3,"Durante a Guerra pela Independência do Sul, GE...",pos,1
3,4,É fora de questão que a verdadeira Anna Anders...,pos,1
4,5,Concordo totalmente com outro dos revisores aq...,neg,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44514 entries, 0 to 44513
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   codigo                 44514 non-null  int64 
 1   texto                  44514 non-null  object
 2   sentimento             44514 non-null  object
 3   sentimento_codificado  44514 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.4+ MB


In [9]:
df['texto_processado'] = df['texto'].apply(txt_handler.tokenizer)

In [10]:
from sklearn.model_selection import train_test_split

## Separação do dataset em teste e treino

In [11]:
X = df['texto_processado'].copy()
y = df['sentimento_codificado'].copy()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Utilizando CountVectorizer para extração de features do texto

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,2)) 
vect.fit(X_train)
X_train_count_vec = vect.transform(X_train)
X_train_count_vec

<35611x2415006 sparse matrix of type '<class 'numpy.int64'>'
	with 7772018 stored elements in Compressed Sparse Row format>

In [14]:
X_test_count_vec = vect.transform(X_test)
X_test_count_vec

<8903x2415006 sparse matrix of type '<class 'numpy.int64'>'
	with 1455957 stored elements in Compressed Sparse Row format>

In [15]:
print(X_train_count_vec.A)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Utilizando TfidfVectorizer para extração de features do texto

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_id_vec = TfidfVectorizer(lowercase=False,ngram_range=(1,1))
tf_id_vec.fit(X_train)
X_train_id_vec = tf_id_vec.transform(X_train)

X_test_id_vec = tf_id_vec.transform(X_test)

## Seleção de modelos

In [17]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

In [18]:
clf1 = RandomForestClassifier(max_depth=8, random_state=42)
clf2 = GradientBoostingClassifier(learning_rate=0.01, random_state=42)
clf3 = AdaBoostClassifier(random_state=42)
clf4 = LogisticRegression(random_state=42)

In [19]:
eclf1 = VotingClassifier(estimators=[('rf', clf1), ('gbc', clf2), ('abc', clf3), ('lr', clf4)], voting='hard')

In [20]:
eclf1.fit(X_train_id_vec, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=8,
                                                     random_state=42)),
                             ('gbc',
                              GradientBoostingClassifier(learning_rate=0.01,
                                                         random_state=42)),
                             ('abc', AdaBoostClassifier(random_state=42)),
                             ('lr', LogisticRegression(random_state=42))])

Função compare_models criada para comparar modelos dinamicamente 

In [21]:
from sklearn.metrics import classification_report, f1_score


def compare_models(X_train, y_train, X_test, y_test, models = []):
  for clf in models:
    clf.fit(X_train, y_train)
    y_hat = clf.predict(X_test)
    cr = classification_report(y_test, y_hat)
    print("Classificador: {} \n {}".format(clf.__class__.__name__, cr))
    print("F1 SCORE: ", f1_score(y_test, y_hat, average='weighted'))

## Comparação de modelos utilizando CountVectorizer

In [22]:
compare_models(X_train_count_vec, y_train, X_test_count_vec, y_test, [clf1, clf2, clf3, clf4, eclf1])

Classificador: RandomForestClassifier 
               precision    recall  f1-score   support

           0       0.80      0.76      0.78      4470
           1       0.77      0.81      0.79      4433

    accuracy                           0.79      8903
   macro avg       0.79      0.79      0.79      8903
weighted avg       0.79      0.79      0.79      8903

F1 SCORE:  0.7872615212903785
Classificador: GradientBoostingClassifier 
               precision    recall  f1-score   support

           0       0.83      0.48      0.60      4470
           1       0.63      0.90      0.74      4433

    accuracy                           0.69      8903
   macro avg       0.73      0.69      0.67      8903
weighted avg       0.73      0.69      0.67      8903

F1 SCORE:  0.6724572668853234
Classificador: AdaBoostClassifier 
               precision    recall  f1-score   support

           0       0.83      0.75      0.79      4470
           1       0.77      0.84      0.80      4433

  

/opt/anaconda3/envs/CustomEnvironments/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classificador: LogisticRegression 
               precision    recall  f1-score   support

           0       0.90      0.89      0.89      4470
           1       0.89      0.90      0.90      4433

    accuracy                           0.89      8903
   macro avg       0.90      0.90      0.89      8903
weighted avg       0.90      0.89      0.89      8903

F1 SCORE:  0.8949727093986474


/opt/anaconda3/envs/CustomEnvironments/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classificador: VotingClassifier 
               precision    recall  f1-score   support

           0       0.86      0.85      0.86      4470
           1       0.85      0.86      0.86      4433

    accuracy                           0.86      8903
   macro avg       0.86      0.86      0.86      8903
weighted avg       0.86      0.86      0.86      8903

F1 SCORE:  0.8567899725926661


## Comparação de modelos utilizando TfidfVectorizer

In [23]:
compare_models(X_train_id_vec, y_train, X_test_id_vec, y_test, [clf1, clf2, clf3, clf4, eclf1])

Classificador: RandomForestClassifier 
               precision    recall  f1-score   support

           0       0.82      0.79      0.81      4470
           1       0.80      0.82      0.81      4433

    accuracy                           0.81      8903
   macro avg       0.81      0.81      0.81      8903
weighted avg       0.81      0.81      0.81      8903

F1 SCORE:  0.8074564310040091
Classificador: GradientBoostingClassifier 
               precision    recall  f1-score   support

           0       0.82      0.53      0.64      4470
           1       0.65      0.88      0.75      4433

    accuracy                           0.71      8903
   macro avg       0.74      0.71      0.70      8903
weighted avg       0.74      0.71      0.70      8903

F1 SCORE:  0.6959015312278457
Classificador: AdaBoostClassifier 
               precision    recall  f1-score   support

           0       0.82      0.77      0.79      4470
           1       0.78      0.83      0.80      4433

  

## Texto Lematizado

In [24]:
# df['texto_lematizado'] = df['texto_processado'].apply(txt_handler.lemmatizer)

In [25]:
# df.to_csv('df_completo.csv') # Salvar a versão lematizada em .csv porque demora muito para processar

In [26]:
df_lematizado = pd.read_csv('df_completo.csv')

In [27]:
df_lematizado.head()

,Unnamed: 0,codigo,texto,sentimento,sentimento_codificado,texto_processado,texto_lematizado
0,0,1,Esse bocejo de pia de cozinha de orçamento mui...,neg,0,bocejo pia cozinha orcamento baixo tipo filme ...,bocejo pia cozinhar orcamento baixo tipo filme...
1,1,2,O Bravo parece indicar que o personagem princi...,neg,0,bravo parece indicar personagem principal clar...,bravo parecer indicar personagem principal cla...
2,2,3,"Durante a Guerra pela Independência do Sul, GE...",pos,1,durante guerra independencia sul general spank...,durante guerra independencia sul general spank...
3,3,4,É fora de questão que a verdadeira Anna Anders...,pos,1,questao verdadeira anna anderson nao princesa ...,questao verdadeira anna anderson nao princesa ...
4,4,5,Concordo totalmente com outro dos revisores aq...,neg,0,concordo totalmente outro revisores aqui ficou...,concordo totalmente outro revisores aqui ficar...


### Separação do dataset em teste e treino

In [28]:
X_lematizado = df_lematizado['texto_lematizado'].copy()
y_2 = df_lematizado['sentimento_codificado'].copy()

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_lematizado, y_2, test_size=0.2, random_state=42)

### Utilizando CountVectorizer para extração de features do texto

In [30]:
vect = CountVectorizer(ngram_range=(1,2)) 
vect.fit(X_train)
X_train_count_vec_lemma = vect.transform(X_train)
X_test_count_vec_lemma = vect.transform(X_test)

### Utilizando CountVectorizer para extração de features do texto

In [31]:
tf_id_vec = TfidfVectorizer(lowercase=False, ngram_range=(1,1))
tf_id_vec.fit(X_train)
X_train_id_vec_lemma = tf_id_vec.transform(X_train)
X_test_id_vec_lemma = tf_id_vec.transform(X_test)

### Seleção de modelos

In [32]:
rf = RandomForestClassifier(max_depth=8, random_state=42)
gbc = GradientBoostingClassifier(learning_rate=0.01, random_state=42)
abc = AdaBoostClassifier(random_state=42)
ls = LogisticRegression(random_state=42)

### Comparação de modelos utilizando CountVectorizer

In [33]:
compare_models(X_train_count_vec_lemma, y_train, X_test_count_vec_lemma, y_test, [rf, gbc, abc, ls])

Classificador: RandomForestClassifier 
               precision    recall  f1-score   support

           0       0.82      0.79      0.80      4470
           1       0.79      0.82      0.81      4433

    accuracy                           0.80      8903
   macro avg       0.81      0.80      0.80      8903
weighted avg       0.81      0.80      0.80      8903

F1 SCORE:  0.8047444377243197
Classificador: GradientBoostingClassifier 
               precision    recall  f1-score   support

           0       0.82      0.49      0.61      4470
           1       0.63      0.90      0.74      4433

    accuracy                           0.69      8903
   macro avg       0.73      0.69      0.68      8903
weighted avg       0.73      0.69      0.68      8903

F1 SCORE:  0.6763603240498278
Classificador: AdaBoostClassifier 
               precision    recall  f1-score   support

           0       0.82      0.75      0.78      4470
           1       0.77      0.84      0.80      4433

  

/opt/anaconda3/envs/CustomEnvironments/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Comparação de modelos utilizando TfidfVectorizer

In [34]:
compare_models(X_train_id_vec_lemma, y_train, X_test_id_vec_lemma, y_test, [rf, gbc, abc, ls])

Classificador: RandomForestClassifier 
               precision    recall  f1-score   support

           0       0.83      0.79      0.81      4470
           1       0.80      0.83      0.82      4433

    accuracy                           0.81      8903
   macro avg       0.81      0.81      0.81      8903
weighted avg       0.81      0.81      0.81      8903

F1 SCORE:  0.8120076376495514
Classificador: GradientBoostingClassifier 
               precision    recall  f1-score   support

           0       0.82      0.54      0.65      4470
           1       0.65      0.88      0.75      4433

    accuracy                           0.71      8903
   macro avg       0.74      0.71      0.70      8903
weighted avg       0.74      0.71      0.70      8903

F1 SCORE:  0.6992092442217015
Classificador: AdaBoostClassifier 
               precision    recall  f1-score   support

           0       0.82      0.76      0.79      4470
           1       0.78      0.83      0.80      4433

  

## Conclusão

Tivemos uma leve melhora na acurácia e F1-score dos modelos que utilizaram as features no formato TfidfVectorizer, porém, a Regressão Logistica teve a acurácia e f1-score de aproximadamente 90% utilizando CountVectorizer. Entre utilizar lematização, ou não, não fez tanta diferença na performance dos modelos escolhidos.